In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required libraries
from elasticsearch_dsl import Index, Document, Integer, Text, analyzer, Keyword, Double
from elasticsearch_dsl.connections import connections
from elasticsearch import Elasticsearch, helpers

from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from indexer import ingest_data
from indexer import QA
import logging

In [4]:
# get list of query answer pairs

query_answer_pairs_filepath = '../../../BERT-FAQ/data/StackFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pairs_filepath)
faq_qa_pair_df = relevance_label_df[relevance_label_df['query_type'] == 'faq']
faq_qa_pairs = faq_qa_pair_df.T.to_dict().values()

In [5]:
# check relevance_label_df
relevance_label_df

,id,question,answer,query_type,label
0,1,How to make font strikethrough on github.,Use ~~tildes around the words~~.,user_query,1
1,2,Is it possible to get strikethrough letter fo...,Use ~~tildes around the words~~.,user_query,1
2,3,Making the text on github crossed out.,Use ~~tildes around the words~~.,user_query,1
3,4,Introducing stikethrough formatting on markdow...,Use ~~tildes around the words~~.,user_query,1
4,5,The <s> tag for font on github markdown doesnt...,Use ~~tildes around the words~~.,user_query,1
...,...,...,...,...,...
7898,7899,How do I get an m dash in google docs?,"On Mac, just hold fn + option + -",user_query,1
7899,7900,Do google docs support the longer style dash (...,"On Mac, just hold fn + option + -",user_query,1
7900,7901,I desperately need the em dash in google docs ...,"On Mac, just hold fn + option + -",user_query,1
7901,7902,"I require the longer (em) dash in google docs,...","On Mac, just hold fn + option + -",user_query,1


In [6]:
# check faq_qa_pair_df
faq_qa_pair_df

,id,question,answer,query_type,label
10,11,Strikethrough with GitHub Markdown,Use ~~tildes around the words~~.,faq,1
21,22,Strikethrough with GitHub Markdown,I just used the following syntax <del> ... </...,faq,1
32,33,How do I delete my Facebook account?,from here : Go here: http://www.facebook.com...,faq,1
43,44,How do I delete my Facebook account?,"Go to ""Your Account"". Go to ""Deactivate Accou...",faq,1
54,55,How do I delete my Facebook account?,It is not neccessary to go through the deactiv...,faq,1
...,...,...,...,...,...
7858,7859,Can I trust dropbox and similar services? [clo...,Not sure about Dropbox. But from the Wuala ser...,faq,1
7869,7870,How to create an em dash in Google Drive (Goog...,"Actually, in Google Docs you can have it autom...",faq,1
7880,7881,How to create an em dash in Google Drive (Goog...,"Answer by @SamtheBrand is correct, but there a...",faq,1
7891,7892,How to create an em dash in Google Drive (Goog...,Insert > Special Characters... Make sure you...,faq,1


In [7]:
# convert to list
faq_qa_pairs = list(faq_qa_pairs)

In [8]:
# check first 5 entries in faq_qa_pairs
faq_qa_pairs[:5]

[{'id': '11',
  'question': 'Strikethrough with GitHub Markdown',
  'answer': 'Use ~~tildes around the words~~.',
  'query_type': 'faq',
  'label': 1},
 {'id': '22',
  'question': 'Strikethrough with GitHub Markdown',
  'answer': 'I just used the following syntax  <del> ... </del>   successfully on GitHub in an issue description.',
  'query_type': 'faq',
  'label': 1},
 {'id': '33',
  'question': 'How do I delete my Facebook account?',
  'answer': 'from here :   Go here: http://www.facebook.com/help/delete_account  Click "Submit" and follow the instructions.  Your account will be deactivated for two weeks, and if you DO NOT USE FACEBOOK IN ANY WAY during that period, your account is permanently deleted.',
  'query_type': 'faq',
  'label': 1},
 {'id': '44',
  'question': 'How do I delete my Facebook account?',
  'answer': 'Go to "Your Account".  Go to "Deactivate Account".  Facebook is a megalomaniac.  It will never ever ever EVER delete your profile.  If you decide ten years later to s

In [9]:
try:
    
    es = connections.create_connection(hosts=['localhost'])
    
    # Index data to Elasticsearch 
    index_name = "stackfaq"
    
    # Initialize index (only perform once)
    index = Index(index_name)

    # Define custom settings
    index.settings(
        number_of_shards=1,
        number_of_replicas=0
    )

    # Delete the index, ignore if it doesn't exist
    index.delete(ignore=404)

    # Create the index in Elasticsearch
    index.create()

    # Register a document with the index
    index.document(QA)

    # Ingest data to Elasticsearch
    ingest_data(faq_qa_pairs, es=es, index=index_name)

    print("Finished indexing {} records to {} index".format(len(faq_qa_pairs), index_name))

except Exception:
    logging.error('exception occured', exc_info=True)


100%|██████████| 719/719 [00:00<00:00, 24703.50it/s]


Finished indexing 719 records to stackfaq index
